In [3]:
%pip install -qU pypdf

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\Programming-Projects\Hackathon\hackrx6\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [29]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

In [5]:
file_path = "docs/BAJHLIP23020V012223.pdf"

In [6]:
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [7]:
len(pages)

49

In [8]:
pages[0].page_content

'UIN- BAJHLIP23020V012223                                 Global Health Care/ Policy Wordings/Page 1 \n \n \nBajaj Allianz General Insurance Co. Ltd.                       \nBajaj Allianz House, Airport Road, Yerawada, Pune - 411 006. Reg. No.: 113 \nFor more details, log on to: www.bajajallianz.com | E-mail: bagichelp@bajajallianz.co.in or \nCall at: Sales - 1800 209 0144 / Service - 1800 209 5858 (Toll Free No.) \nIssuing Office: \n \nGLOBAL HEALTH CARE \n \n \nPolicy Wordings \n \nUIN- BAJHLIP23020V012223 \nSECTION A) PREAMBLE \n \nWhereas the Insured described in the Policy Schedule hereto (hereinafter called the ‘Insured’  or “Policyholder” or \n“Insured Person”) has made to Bajaj Allianz General Insurance Company Limited (hereinafter called the “Company” \nor “Insurer” or “Insurance Company”) a proposal or Proposal as mentioned in the transcript of the Proposal, which \nshall be the basis of this Contract and is deemed to be incorporated herein, containing certain undertakings , 

In [19]:
len(pages[0].page_content)

4500

In [27]:
# Splitting into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

In [28]:
len(splits[0].page_content)

978

In [40]:
# Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
all_mini_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
bge_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
embeddings = all_mini_embeddings
vector_store = FAISS.from_documents(splits, embedding=embeddings)

In [47]:
# Store the vector DB locally
vector_store.save_local("vector_store/faiss_index_bajaj")

In [ ]:
# Load the saved vector store
vector_store = FAISS.load_local("vector_store/faiss_index_bajaj", embeddings, allow_dangerous_deserialization=True)

In [42]:
retriever = vector_store.as_retriever()

In [38]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from langchain_core.runnables import RunnableLambda

def log_chunks(docs):
    print("\nRetrieved Chunks:\n" + "\n---\n".join(doc.page_content for doc in docs))
    return docs

# Modified retriever pipeline
logged_retriever = retriever | RunnableLambda(log_chunks) | format_docs

In [56]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub


prompt = hub.pull("rlm/rag-prompt")

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": logged_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
# rag_chain.invoke("what is the minimum and maximum In-patient Hospitalization Treatment Limits for Imperial plus plan?")
# rag_chain.invoke("46-year-old male, knee surgery in Pune, 3-month-old insurance policy")
rag_chain.invoke("knee surgery?")


🔍 Retrieved Chunks:
75  Rotational Arc Therapy 275  Testicular biopsy  
76   Tele gamma therapy 276  laparoscopic cardiomyotomy( Hellers) 
77  FSRT-Fractionated SRT 277  Sentinel node biopsy malignant melanoma 
78  VMAT-Volumetric Modulated Arc Therapy 278   laparoscopic pyloromyotomy( Ramstedt) 
79  SBRT-Stereotactic Body Radiotherapy Orthopedics 
80  Helical Tomotherapy 279  Arthroscopic Repair of ACL tear knee 
81  SRS-Stereotactic Radiosurgery 280  Closed reduction of minor Fractures 
82  X-Knife SRS 281  Arthroscopic repair of PCL tear knee 
83  Gammaknife SRS 282  Tendon shortening 
84  TBI- Total Body Radiotherapy 283  Arthroscopic  Meniscectomy - Knee 
85  intraluminal Brachytherapy 284  Treatment of clavicle dislocation 
86   Electron Therapy 285  Arthroscopic meniscus repair 
87  TSET-Total Electron Skin Therapy 286  Haemarthrosis knee- lavage 
88  Extracorporeal Irradiation of  Blood Products 287  Abscess  knee joint drainage  
89  Telecobalt Therapy 288  Carpal tunnel rele

'Knee surgeries include arthroscopic repair of ACL or PCL tears, arthroscopic meniscectomy or meniscus repair, haemarthrosis knee lavage, abscess knee joint drainage, and repair of the knee cap tendon. Other knee surgeries are fixation, repair of the knee joint, and removal of the knee cap bursa.'